In [1]:
import pandas as pd

In [5]:
class DPLLSolver:
    def __init__(self, clauses):
        self.clauses = clauses  # Store original problem clauses
        self.assignment = {}  # Track variable assignments
        self.decided = {}  # Track decisions for backtracking
        self.logs = []  # Store logs for table output
        self.iteration = 0  # Track step count

    # Step 1: Define/Update Current Clauses
    def define_current_clauses(self):
        return self.clauses

    # Step 2: Unit Propagation
    def perform_unit_propagation(self):
        found_literals = []
        for clause in self.clauses:
            if len(clause) == 1:  # Check for unit clause
                literal = list(clause)[0]
                if literal.startswith("¬"):
                    symbol = literal[1:]  # Remove "¬" to get the variable name
                    self.assignment[symbol] = False
                else:
                    self.assignment[literal] = True
                found_literals.append(literal)

        return found_literals if found_literals else None  # Return found literals

    # Step 3: Pure Literal Elimination
    def pure_literal_elimination(self):
        literal_counts = {}  # Track occurrences of each literal

        for clause in self.clauses:
            for literal in clause:
                literal_counts[literal] = literal_counts.get(literal, 0) + 1

        eliminated_pure_literals = {}
        for literal in literal_counts:
            if literal.startswith("¬"):
                potential_pure_literal = literal[1:]
                if potential_pure_literal not in literal_counts:
                    eliminated_pure_literals[potential_pure_literal] = False
            else:
                potential_pure_literal = literal
                negated = f"¬{potential_pure_literal}"
                if negated not in literal_counts:
                    eliminated_pure_literals[potential_pure_literal] = True

        found_literals = []
        for symbol, value in eliminated_pure_literals.items():
            self.assignment[symbol] = value
            found_literals.append(f"{symbol}={value}")

        return found_literals if found_literals else None

    # Step 4: Organise Assignment
    def organise_assignment(self):
        new_assignments = {}
        original_literals = set()

        for clause in self.clauses:
            for literal in clause:
                original_literals.add(literal.lstrip("¬"))

        for symbol, value in self.assignment.items():
            if symbol not in original_literals:  # Skip if not in original clauses
                continue

            if value is False and f"¬{symbol}" in original_literals:
                new_assignments[f"¬{symbol}"] = True
            elif value is True and f"¬{symbol}" in original_literals:
                new_assignments[f"¬{symbol}"] = False

        self.assignment.update(new_assignments)
        return self.assignment

    # Step 5: Decision Making
    def decision(self):
        for clause in self.clauses:
            for literal in clause:
                symbol = literal.lstrip("¬")
                if symbol not in self.assignment:
                    self.assignment[symbol] = True
                    self.decided[symbol] = True
                    return symbol, True  # Decision made

        return None, None  # No more decisions needed

    # Step 6: Check for Logic Conflict
    def check_conflict(self):
        for clause in self.clauses:
            if all(literal.lstrip("¬") in self.assignment for literal in clause):
                if not any((literal.startswith("¬") and not self.assignment.get(literal[1:], True)) or  # Changed line
                          (not literal.startswith("¬") and self.assignment.get(literal, False)) # Changed line
                          for literal in clause):
                    return True  # Conflict detected

        if all(any((literal.startswith("¬") and not self.assignment.get(literal[1:], True)) or  # Changed line
                  (not literal.startswith("¬") and self.assignment.get(literal, False)) # Changed line
                  for literal in clause) for clause in self.clauses):
            return "SAT"

        return False  # No conflict

    # Step 7: Backtrack
    def backtrack(self):
        if not self.decided:
            return "UNSAT", None  # Return a tuple of 2 values

        last_decided = list(self.decided.keys())[-1]
        if self.assignment[last_decided] is True:
            self.assignment[last_decided] = False
            self.decided[last_decided] = False
            return last_decided, False  # Return 2 values
        else:
            del self.assignment[last_decided]
            del self.decided[last_decided]
            return self.backtrack()  # Recursive call

    # Run the SAT Solver
    def solve(self):
        while True:
            self.iteration += 1
            unit_literals = self.perform_unit_propagation()
            pure_literals = self.pure_literal_elimination()
            self.organise_assignment()

            decision_var, decision_value = self.decision()
            conflict = self.check_conflict()

            backtracked_var, backtracked_value = None, None
            if conflict == "SAT":
                self.log_result("SAT")
                return "SAT"
            elif conflict:
                backtracked_var, backtracked_value = self.backtrack()
                if backtracked_var == "UNSAT":
                    self.log_result("UNSAT")
                    return "UNSAT"

            # Log the iteration
            self.logs.append([
                self.iteration,
                unit_literals,
                pure_literals,
                f"{decision_var}={decision_value}" if decision_var else None,
                f"{backtracked_var}→{backtracked_value}" if backtracked_var else None,
                None  # Final result not determined yet
            ])


    def log_result(self, result):
        # Mark the last row as final result
        if self.logs:
            self.logs[-1][-1] = result

        df = pd.DataFrame(self.logs, columns=[
            "Time", "Unit Propagation", "Pure Literal Elimination", "Decision", "Backtracking", "Final Result"
        ])

        print("\n============================== DPLL SAT Solver Steps ==============================")
        print(df.to_string(index=False))  # Display formatted table
        print("=====================================================================================\n")

In [7]:
# Example Usage
clauses = [{"X", "Y", "Z"}, {"X", "¬Y", "¬Z"}, {"¬X", "Y", "Z"},
           {"¬X", "¬Y", "¬Z"}, {"¬Y", "Z"}, {"Y", "¬Z"}]

solver = DPLLSolver(clauses)
result = solver.solve()
print("Final Result:", result)


============================== DPLL SAT Solver Steps ==============================
 Time Unit Propagation Pure Literal Elimination Decision Backtracking Final Result
    1             None                     None   X=True         None         None
    2             None                     None   Y=True         None         None
    3             None                     None   Z=True      Z→False         None
    4             None                     None     None      Y→False         None
    5             None                     None   Z=True      Z→False         None
    6             None                     None     None      X→False         None
    7             None                     None   Y=True         None         None
    8             None                     None   Z=True      Z→False         None
    9             None                     None     None      Y→False         None
   10             None                     None   Z=True      Z→False        UNSAT

F

# EOF